In [ ]:
# Importing libraries
import nltk
stopwords = nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import pandas as pd

from nltk import tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#Loading and cleaning dataset
df1 = pd.read_excel('/content/drive/My Drive/data/covid1.xlsx')
#df_summary = pd.DataFrame(columns = ['Summary']) 
df=df1.drop('All_content',axis=1)
df = df.drop('Unnamed: 0',axis=1)
df = df.dropna().drop_duplicates().reset_index(drop=True)

In [ ]:
df.head()

,Date-Time,Title,Content,Article URL,All_Content
0,"21 Apr, 2020, 01:52PM IST",Covid-19: Karnataka to collect samples of jour...,The directions followed a request from Educati...,https://economictimes.indiatimes.com//news/pol...,After 53 journalists in Mumbai were found posi...
1,"20 Apr, 2020, 07:33AM IST",Newborn tests positive for COVID-19 in Rajasth...,"Dr Shadab Ali, in-charge of Basni primary heal...",https://economictimes.indiatimes.com//news/pol...,Jaipur: A newborn baby has tested positive for...
2,"18 Apr, 2020, 12:17PM IST",Police officer dies of COVID-19 in Ludhiana,The 52-year-old Ludhiana assistant commissione...,https://economictimes.indiatimes.com//news/pol...,The 52-year-old Ludhiana assistant commissione...
3,"10 Apr, 2020, 02:14AM IST",The Covid curve: How the states fare,Data suggests that some of the 15 states/UTs n...,https://economictimes.indiatimes.com//news/pol...,Fifteen Indian states and Union Territories ac...
4,"15 Apr, 2020, 06:33PM IST",Covid fight needs women to be agents of change,"Women civil servants and police at Centre, sta...",https://economictimes.indiatimes.com//news/pol...,"By LAKSHMI PURI In our Covid wars, it’s time t..."


In [ ]:
# Creating an empty Summary attribute in the dataset
df['Summary'] = ''

In [ ]:
df.head()

,Date-Time,Title,Content,Article URL,All_Content,Summary
0,"21 Apr, 2020, 01:52PM IST",Covid-19: Karnataka to collect samples of jour...,The directions followed a request from Educati...,https://economictimes.indiatimes.com//news/pol...,After 53 journalists in Mumbai were found posi...,
1,"20 Apr, 2020, 07:33AM IST",Newborn tests positive for COVID-19 in Rajasth...,"Dr Shadab Ali, in-charge of Basni primary heal...",https://economictimes.indiatimes.com//news/pol...,Jaipur: A newborn baby has tested positive for...,
2,"18 Apr, 2020, 12:17PM IST",Police officer dies of COVID-19 in Ludhiana,The 52-year-old Ludhiana assistant commissione...,https://economictimes.indiatimes.com//news/pol...,The 52-year-old Ludhiana assistant commissione...,
3,"10 Apr, 2020, 02:14AM IST",The Covid curve: How the states fare,Data suggests that some of the 15 states/UTs n...,https://economictimes.indiatimes.com//news/pol...,Fifteen Indian states and Union Territories ac...,
4,"15 Apr, 2020, 06:33PM IST",Covid fight needs women to be agents of change,"Women civil servants and police at Centre, sta...",https://economictimes.indiatimes.com//news/pol...,"By LAKSHMI PURI In our Covid wars, it’s time t...",


In [ ]:
# Reading the data
def read_article(file_name):
    a = file_name

    #article = a.split(".")
    article = tokenize.sent_tokenize(a)

    sentences = []

    for sentence in article:
        #print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop() 
    
    return sentences

# Defining a function depicting sentence similarity
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

# Generating the summary
def generate_summary(file_name, top_n=10):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text anc split it
    sentences =  read_article(file_name)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank_numpy(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    #print("Indexes of top ranked_sentence order are ", ranked_sentence)    

    for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Offcourse, output the summarize text
    #print("Summarize Text: \n", ". ".join(summarize_text))
    return summarize_text

def listToString(s):  
    
    # initialize an empty string 
    str1 = ""  
    
    # traverse in the string   
    for ele in s:  
        str1 += ele   
    
    # return string   
    return str1  


# let's begin
for index,row in df.iterrows():
  string2 = str(row['All_Content'])
  ## Data cleaning
  a=string2
  #a=a.replace('', '')
  a=a.replace('.   ', '. ')
  a=a.replace('.  ', '. ')
  a=a.replace('. ', '. ')
  a=a.replace('  ', ' ')
  a=a.replace('   ', ' ')
  try:
    # Trying to generate summary of 5 sentences
    summary = generate_summary(a, 5)
    
  except:
    # Else trying to generate summary of 2 sentences for data rows consisting of a few sentences.
    summary = generate_summary(a, 2)

  summary = listToString(summary)
  # Entering summary data into the newly made Summary feature column
  df.at[index,'Summary'] = summary


In [ ]:
# Exporting the file which contains Summary attribute too
from google.colab import files
df.to_excel('covid_summary.xlsx')